In [ ]:
import networkx as nx
import nxpd
import string
import math
import graphviz as gv


In [ ]:
G = nx.DiGraph()
G.graph['dpi'] = 120
G.add_nodes_from(range(1,9))
G.add_edges_from([(1,2),(1,3),(2,4),(3,6),(4,5),(4,6),(5,7),(5,8)])
nxpd.draw(G, show='ipynb')

In [ ]:
g_ordered = nx.DiGraph()
g_ordered

In [ ]:
list(string.ascii_lowercase)[:13] + ['r']

In [ ]:
type(string.ascii_lowercase) is str

In [ ]:
def add_edges_path(graph1, vertices_path):
    if vertices_path is str:
        vertices_path = list(vertices_path)
    for ix in range(len(vertices_path)-1):
        graph1.add_edge(vertices_path[ix],vertices_path[ix+1])

In [ ]:
G1 = nx.DiGraph()
G.graph['dpi'] = 60
G1.add_nodes_from(list(string.ascii_lowercase)[:13] + ['r'])
add_edges_path(G1,'rm')
add_edges_path(G1,'bfgkl')
add_edges_path(G1,'r'+ string.ascii_lowercase[:5])
add_edges_path(G1,'ghi')
add_edges_path(G1,'hj')
nxpd.draw(G1, show='ipynb')

In [ ]:
G1.adj

In [ ]:
def tee_at(g1, node):
    g2 = nx.DiGraph()
    list_nodes = [node]
    while(len(list_nodes)>0):
        node_current = list_nodes.pop()
        if not g2.has_node(node_current):
            g2.add_node(node_current)
        for neighbor in g1.adj[node_current].keys():
            g2.add_edge(node_current, neighbor)
            list_nodes.append(neighbor)
    return g2

In [ ]:
g2_at_f = tee_at(G1, 'f')
nxpd.draw(g2_at_f, show='ipynb')

In [ ]:
def find_path(g2, node1, node2):
    if node1==node2:
        return [node1]
    
    for child in g2.adj[node1].keys():
        result = find_path(g2, child, node2)
        if len(result)>0:
            return [node1] + result
    return []
        
print(find_path(g2_at_f, 'f','i'))
    

In [ ]:
def path_to_edges(path1):
    return [ (path1[ix],path1[ix+1]) for ix in range(len(path1)-1)]
path1 = path_to_edges(find_path(g2_at_f, 'f', 'i'))
print(path1)
for (a,b) in path1:
    print(a,b)

In [ ]:
def make_parent(g1, node_parent, node_child):
    # remove the child -> parent link if present
    if node_parent in g1[node_child]:
        g1.remove_edge(node_child, node_parent)
        
    # link node_child's parents to node_parent
    print(g1.predecessors(node_child))
    for predecessor in g1.predecessors(node_child):
        print(predecessor)
        g1.remove_edge(predecessor, node_child)
        g1.add_edge(predecessor, node_parent)
        
    # make parent -> child link
    g1.add_edge(node_parent, node_child)
        
g_make_parent = nx.DiGraph()
g_make_parent.add_edge('a','b')
g_make_parent.add_edge('b','c')
print(g_make_parent.adj)
print(g_make_parent.predecessors('c'))
make_parent(g_make_parent, 'c', 'b')
print(g_make_parent.adj)

In [ ]:
g_make_parent

In [ ]:
def reroot(g1, r0, r_prime):
    path1 = find_path(g1, r0, r_prime)
    for (a,b) in path_to_edges(path1):
        make_parent(g1, b, a)
        # for each child c of b
        for c in g1.adj[b]:
            # c is in path(r0, r')
            if c in path1:
                continue
            result1 = query(g1, c, r0, b)
            if result1 is None:
                continue
            (u, v) = result1
            reroot(g1, c, v)
            make_parent(u, v)
            
            
        

Making the data structure
===============================

emulate figure 5

In [ ]:
help(nx.depth_first_search.dfs_tree)

In [ ]:
g2 = nx.Graph()
add_edges_path(g2, 'xzws')
add_edges_path(g2, 'xuv')
add_edges_path(g2, 'zy')
add_edges_path(g2, 'wt')
add_edges_path(g2, 'xs')
add_edges_path(g2, 'zt')
nxpd.draw(g2, show='ipynb')

In [ ]:
a = [1,2]
a.append(3)
print(a.pop())

In [ ]:
's' in g2_dfs.nodes()

In [ ]:
def form_dfs_tree_slave(g, g_dfs, node, visited):
    if node in visited:
        return
    visited.append(node)
    children = sorted(g.adj[node].keys())
    children.reverse()
    for child in children:
        if child in visited:
            continue
        g_dfs.add_edge(node, child)
        form_dfs_tree_slave(g, g_dfs, child, visited)
    
def form_dfs_tree(g, source):
    g_dfs = nx.DiGraph()
    form_dfs_tree_slave(g, g_dfs, source, [])
    return g_dfs
    
nxpd.draw(form_dfs_tree(g2,'x'), show='ipynb')

In [ ]:
g2_dfs = nx.depth_first_search.dfs_tree(g2,source='x')

In [ ]:
type(g2.adj['x'])

In [ ]:
nxpd.draw(g2_dfs, show='ipynb')

In [ ]:
['x', 'z', 't', 'w', 's', 'y', 'u', 'v']
g2_dfs.__dict__

Heavy-light decomposition
--------------------------

step 0: find the size of each node in the dfs

In [ ]:
def find_root_dfs(g_dfs):
   return [child for (child,parent) in g_dfs.pred.items() if not parent][0]

def subtree_sizes(g_dfs, root_dfs=None):
    if root_dfs is None:
        root_dfs = find_root_dfs(g_dfs)
    dict_1 = {root_dfs: 1}
    for child in g_dfs.adj[root_dfs].keys():
        dict_child = subtree_sizes(g_dfs, child)
        if not dict_child:
            continue
        dict_1[root_dfs] += dict_child[child]
        dict_1.update(dict_child)
    return dict_1

subtree_sizes(g2_dfs)

step 1: form the preorder traversal list

In [ ]:
def preorder_heavy_light(g_dfs, root_dfs=None, g_dfs_sizes=None):
    if root_dfs is None:
        root_dfs = find_root_dfs(g_dfs)
    if g_dfs_sizes is None:
        g_dfs_sizes = subtree_sizes(g_dfs, root_dfs)
    list1 = [root_dfs]
    children = list(g_dfs.adj[root_dfs].keys())
    children.sort(key=lambda x: -g_dfs_sizes[x])
    print(root_dfs, children)
    for child in children:
        list1 += preorder_heavy_light(g_dfs, child, g_dfs_sizes)
    return list1
    
preorder_heavy_light(g2_dfs)

In [ ]:
help([1,2].sort)

step 2: form the segment tree

In [ ]:
a = list(range(5))
a.reverse()
print(a)
print(a[::2])
print(a[1::2])

In [ ]:
class st_node:
    def __init__(self, name=None, bst=None):
        self.name = name
        self.add_bst(bst)
        self.parent = None
        self.left = None
        self.right = None
    def add_bst(self, bst):
        self.bst = bst
    def make_bst(self, neighbors):
        if self.bst is None:
            self.bst = {}
        for nbr in neighbors:
            self.bst[nbr] = self.name

In [ ]:
# currently 2
def groups_of(list1, n):
    even_list, odd_list = list1[0::2], list1[1::2]
    if len(odd_list)!=len(even_list):
        odd_list.append(None)
    return [[even_list[ix], odd_list[ix]] for ix in range(len(even_list))]


In [ ]:
class seg_tree:
    def __init__(self, g2, root_dfs):
        self.graph = g2
        self.root_dfs = root_dfs
        self.main()
        
    def make_dfs(self):
        make_dfs = form_dfs_tree
        self.dfs = make_dfs(self.graph, self.root_dfs)
        self.dfs_sz = subtree_sizes(self.dfs, self.root_dfs)
    
    def make_list_heavy_light(self):
        self.hl_list = preorder_heavy_light(self.dfs,
                                           self.root_dfs,
                                           self.dfs_sz)
    
    def make_st_leaves(self):
        leaves = []
        for node_name in self.hl_list:
            st_node_leaf = st_node(node_name)
            st_node_leaf.make_bst(self.graph.adj[node_name].keys())
            leaves.append(st_node_leaf)
        return leaves
    
    def st_layerize(self, nodes):
        node_pairs = groups_of(nodes, 2)
        print('layerize', [[[n[0].name,n[1].name] for n in node_pairs]])
        return [self.st_add_parent(node_pair) for node_pair in node_pairs]
    def st_add_parent(self, node_pair):
        str_name = self.conc_name(node_pair)
        st_parent = st_node(str_name)
        [self.st_add_edge(st_parent, child) for child in node_pair]
        return st_parent
    def st_add_edge(self, parent, child):
        if child is None:
            return
        # add edges from child
        self.st_add_edges_from_child(parent, child)
        if parent.left is None:
            parent.left = child
        else:
            parent.right = child
    def st_add_edges_from_child(self, parent, child):
        pass
    def conc_name(self, node_pair):
        str_name =  node_pair[0].name
        str_name += node_pair[1].name if node_pair[1] else ''
        return str_name
    
    def make_st(self):
        nodes = self.make_st_leaves()
        while(len(nodes)>1):
            nodes = self.st_layerize(nodes)
        self.st_root = nodes[0]
    
    def main(self):
        self.make_dfs()
        self.make_list_heavy_light()
        self.make_st()
        
    
    def draw_st(self):
        g = gv.Digraph()
        self.form_st(g, self.st_root)
        return g
        
    def form_st(self, g, node):
        #print(node.name)
        self.form_st_child(g, node, node.left)
        self.form_st_child(g, node, node.right)
    def form_st_child(self, g, node, child):
        if child:
            g.edge(node.name, child.name)
            self.form_st(g, child)
        
        


In [ ]:
t2b = seg_tree(g2, 'x')
#t2b.draw_dfs()
#t2b.segment_tree.node
print('hl_list', t2b.hl_list)
#nxpd.draw(t2b.dfs, show='ipynb')
#nxpd.draw(t2b.segment_tree, show='ipynb')
print(t2b.st_root.name)
t2b.draw_st()

In [ ]:
from drawtree import draw_bst
nums = [55, 30, 10, 5, 2, 20, 15, 25, 40, 35, 70, 60, 80, 75, 95]
draw_bst(nums)

In [ ]:
t2b.segment_tree.node['z']['st_node'].__dict__

In [ ]:
g2.adj['x'].keys()

In [ ]:
nxpd.draw(g2, show='ipynb')

In [ ]:
g2_dfs.nodes()

In [ ]:
help(g2_dfs.add_node)

In [ ]:
a=[1,2,3]
a.sort(f)

In [ ]:
b_hash = {'a': 1, 'b': 2, 'c': 3}
[a for a in b_hash]
list(b_hash.keys())

In [ ]:
b_hash = {'a': 1, 'b': 2, 'c': 3}
print(b_hash)
key_to_remove = 'e'
if key_to_remove in b_hash:
    b_hash.pop(key_to_remove)
print(b_hash)

In [ ]:
len(G1.adj['i'].keys())

In [ ]:
a=[1,2,3]
print(a)
a.pop()
print(a)
a.append(4)
print(a)

In [ ]:
G1.adj